In [20]:
'''Os resultados para este código etão melhores mas ainda longe do ideal'''

'Os resultados para este código etão melhores mas ainda longe do ideal'

In [21]:
# %% [1] Imports e configuração
import os, json, pickle, re, numpy as np
from rank_bm25 import BM25Okapi
from openai import OpenAI
from dotenv import load_dotenv
import faiss

# Carregar variáveis do .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# %% [2] Carregar chunks e embeddings
# Chunks originais (para BM25)
with open(r"D:\Ciencia de Dados\Projeto Integrador de Extenção 3\RAGzilla\project\data\processed\chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# Embeddings + metadados (FAISS)
with open("metadata.pkl", "rb") as f:
    embeddings_list = pickle.load(f)

index = faiss.read_index("embeddings.index")

print(f"✅ {len(chunks)} chunks carregados do JSON")
print(f"✅ {len(embeddings_list)} embeddings carregados")


✅ 68 chunks carregados do JSON
✅ 68 embeddings carregados


In [22]:
import re
import unicodedata

def preprocess(text):
    # lowercase
    text = text.lower()
    
    # remover acentos
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    
    # manter só letras e números (sem pontuação)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # remover espaços extras e tokenizar
    return text.split()


# Tokenizar chunks para BM25
tokenized_chunks = [preprocess(chunk["text"]) for chunk in chunks]

# Criar índice BM25
bm25 = BM25Okapi(tokenized_chunks)
print("✅ Índice BM25 criado")


✅ Índice BM25 criado


In [23]:
# %% [4] Função para gerar embedding da query
import numpy as np

def gerar_embedding(texto):
    # aplica normalização na query
    texto = ' '.join(preprocess(texto))  
    resp = client.embeddings.create(
        model="text-embedding-3-large",
        input=texto
    )
    return np.array(resp.data[0].embedding, dtype="float32")


In [24]:
# %% [5] Função de busca híbrida BM25 + embeddings
def buscar_hibrido(query, top_bm25=100, top_k=5, metric="cosine"):
    # 1️⃣ BM25
    query_tokens = preprocess(query)
    bm25_scores = bm25.get_scores(query_tokens)
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:top_bm25]

    # 2️⃣ Embeddings da query
    q_emb = gerar_embedding(query)
    if metric in ["cosine", "dot"]:
        q_emb = q_emb / np.linalg.norm(q_emb)

    # 3️⃣ Calcular score só nos top_bm25
    scores = []
    for i in bm25_top_idx:
        chunk_emb = np.array(embeddings_list[i]["embedding"])
        if metric == "cosine" or metric == "dot":
            score = np.dot(q_emb, chunk_emb)
        else:  # l2
            score = -np.linalg.norm(q_emb - chunk_emb)
        scores.append((i, score))

    # 4️⃣ Ordena pelo score do embedding
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    top_chunks = [chunks[i]["chunk_number"] for i, s in scores[:top_k]]
    return top_chunks


In [25]:
avaliacoes = [
    {"query": "Qual contrato tem a empresa com sede na Rua Alfred Charvet", "relevante": [0]},  # índices dos chunks relevantes
    {"query": "ANDREY BARRETO MOLINARI COMERCIO", "relevante": [32, 50]},
    {"query": "Qual empresa é representada pelo sócio administrador Jhonny Cézar de Jesus Falavinhacontrato", "relevante": [0]}, 
    {"query": "Jhonny Cézar de Jesus Falavinhacontrato", "relevante": [0, 30, 53]},
    {"query": "Contrato que tenha o CNPJ 76.443.340/0001-59", "relevante": [52]}
]   

for caso in avaliacoes:
    top_chunks = buscar_hibrido(caso["query"], top_bm25=100, top_k=5)
    print(f"Query: {caso['query']}")
    print(f"Top 5 chunk numbers: {top_chunks}\n")

Query: Qual contrato tem a empresa com sede na Rua Alfred Charvet
Top 5 chunk numbers: [27, 66, 0, 49, 56]

Query: ANDREY BARRETO MOLINARI COMERCIO
Top 5 chunk numbers: [27, 45, 66, 50, 26]

Query: Qual empresa é representada pelo sócio administrador Jhonny Cézar de Jesus Falavinhacontrato
Top 5 chunk numbers: [0, 67, 27, 28, 47]

Query: Jhonny Cézar de Jesus Falavinhacontrato
Top 5 chunk numbers: [25, 32, 15, 66, 28]

Query: Contrato que tenha o CNPJ 76.443.340/0001-59
Top 5 chunk numbers: [48, 28, 1, 50, 53]

